In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras

from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras import layers

In [2]:
df = pd.read_csv('car_price_prediction.csv')

In [3]:
df.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


Quick check for NaN values and duplicates.

In [4]:
# print out NaN values in each column
df.isna().sum()

ID                  0
Price               0
Levy                0
Manufacturer        0
Model               0
Prod. year          0
Category            0
Leather interior    0
Fuel type           0
Engine volume       0
Mileage             0
Cylinders           0
Gear box type       0
Drive wheels        0
Doors               0
Wheel               0
Color               0
Airbags             0
dtype: int64

In [5]:
# print amount of duplicate rows
df.duplicated().sum()

313

In [6]:
# detele all duplicate rows
df.drop_duplicates(inplace=True)

In [7]:
# delete ID column, there is no any information there, only identificator
df.drop("ID", inplace=True,axis=1)

<h2>Levy</h2>
Levy in auto dealeship language could mean any additional fees, such as taxes and etc.

This column has "-" value in 30% of dataset, so I assume these mean not the NaN values, but the fact that there are no fees. So, I'll change them to zeroes.

In [8]:
df["Levy"].replace("-", "0", inplace=True)
# change data type of column to numeric
df["Levy"] = pd.to_numeric(df["Levy"])

C:\Users\ksobolev\AppData\Local\Temp\ipykernel_19780\2061335554.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Levy"].replace("-", "0", inplace=True)


<h2>Manufacturer, model</h2>
Manufacturer column has 65 distinct values. My first thought was to leave only let's say 10 mostly popular ones and bucket up everything else in "other" category. But then I had an idea to divide them into countries of origin, let's see if it works. 

I believe that it might work because most of the car brands in average consumer mind are related to country of origin and e.g. all korean or japanese cars are consider as reliable, or german cars well engineered. However I might loose some brands from premium or "very poor" segment. On the other hand they would possibly be outliers anyways, so...

ChatGPT was very usefull in this. Also I joined minority countries such as Georgia, Spain and Czech Republic into temporary "other" category which should be deleted afterwards. Model will automatically understand that if the car is not in any category, then it is "other".

In [9]:
df["Manufacturer"].unique()

array(['LEXUS', 'CHEVROLET', 'HONDA', 'FORD', 'HYUNDAI', 'TOYOTA',
       'MERCEDES-BENZ', 'OPEL', 'PORSCHE', 'BMW', 'JEEP', 'VOLKSWAGEN',
       'AUDI', 'RENAULT', 'NISSAN', 'SUBARU', 'DAEWOO', 'KIA',
       'MITSUBISHI', 'SSANGYONG', 'MAZDA', 'GMC', 'FIAT', 'INFINITI',
       'ALFA ROMEO', 'SUZUKI', 'ACURA', 'LINCOLN', 'VAZ', 'GAZ',
       'CITROEN', 'LAND ROVER', 'MINI', 'DODGE', 'CHRYSLER', 'JAGUAR',
       'ISUZU', 'SKODA', 'DAIHATSU', 'BUICK', 'TESLA', 'CADILLAC',
       'PEUGEOT', 'BENTLEY', 'VOLVO', 'სხვა', 'HAVAL', 'HUMMER', 'SCION',
       'UAZ', 'MERCURY', 'ZAZ', 'ROVER', 'SEAT', 'LANCIA', 'MOSKVICH',
       'MASERATI', 'FERRARI', 'SAAB', 'LAMBORGHINI', 'ROLLS-ROYCE',
       'PONTIAC', 'SATURN', 'ASTON MARTIN', 'GREATWALL'], dtype=object)

In [10]:
# dictionary with manufacters and origin countries according to ChatGPT
car_manufacturers = {
    "Japan": ['LEXUS', 'HONDA', 'TOYOTA', 'NISSAN', 'SUBARU', 'MITSUBISHI', 'MAZDA', 'SUZUKI', 'ACURA', 'ISUZU', 'DAIHATSU', 'INFINITI', 'SCION'],
    "USA": ['CHEVROLET', 'FORD', 'JEEP', 'GMC', 'LINCOLN', 'DODGE', 'CHRYSLER', 'TESLA', 'BUICK', 'CADILLAC', 'HUMMER', 'MERCURY', 'PONTIAC', 'SATURN'],
    "South Korea": ['HYUNDAI', 'KIA', 'DAEWOO', 'SSANGYONG'],
    "Germany": ['MERCEDES-BENZ', 'OPEL', 'PORSCHE', 'BMW', 'VOLKSWAGEN', 'AUDI'],
    "France": ['RENAULT', 'CITROEN', 'PEUGEOT'],
    "UK": ['LAND ROVER', 'MINI', 'JAGUAR', 'BENTLEY', 'ROLLS-ROYCE', 'ROVER', 'ASTON MARTIN', 'MG'],
    "Italy": ['FIAT', 'ALFA ROMEO', 'LANCIA', 'MASERATI', 'FERRARI', 'LAMBORGHINI'],
    "Sweden": ['VOLVO', 'SAAB'],
    "Russia": ['VAZ', 'GAZ', 'UAZ', 'MOSKVICH', 'ZAZ'],
    "China": ['HAVAL', 'GREATWALL'],
    "Other": ['SKODA', 'სხვა', 'SEAT']
}

In [11]:
# Now map a country to each manufacturer

# empty dictionary to store the mappings
manufacturer_to_country = {}

# Loop over each country and its list of manufacturers
for country, manufacturers in car_manufacturers.items():
    # For each manufacturer in the list, assign the country as the value
    for manufacturer in manufacturers:
        manufacturer_to_country[manufacturer] = country

# Now, 'manufacturer_to_country' maps each manufacturer to its corresponding country
print(manufacturer_to_country)

{'LEXUS': 'Japan', 'HONDA': 'Japan', 'TOYOTA': 'Japan', 'NISSAN': 'Japan', 'SUBARU': 'Japan', 'MITSUBISHI': 'Japan', 'MAZDA': 'Japan', 'SUZUKI': 'Japan', 'ACURA': 'Japan', 'ISUZU': 'Japan', 'DAIHATSU': 'Japan', 'INFINITI': 'Japan', 'SCION': 'Japan', 'CHEVROLET': 'USA', 'FORD': 'USA', 'JEEP': 'USA', 'GMC': 'USA', 'LINCOLN': 'USA', 'DODGE': 'USA', 'CHRYSLER': 'USA', 'TESLA': 'USA', 'BUICK': 'USA', 'CADILLAC': 'USA', 'HUMMER': 'USA', 'MERCURY': 'USA', 'PONTIAC': 'USA', 'SATURN': 'USA', 'HYUNDAI': 'South Korea', 'KIA': 'South Korea', 'DAEWOO': 'South Korea', 'SSANGYONG': 'South Korea', 'MERCEDES-BENZ': 'Germany', 'OPEL': 'Germany', 'PORSCHE': 'Germany', 'BMW': 'Germany', 'VOLKSWAGEN': 'Germany', 'AUDI': 'Germany', 'RENAULT': 'France', 'CITROEN': 'France', 'PEUGEOT': 'France', 'LAND ROVER': 'UK', 'MINI': 'UK', 'JAGUAR': 'UK', 'BENTLEY': 'UK', 'ROLLS-ROYCE': 'UK', 'ROVER': 'UK', 'ASTON MARTIN': 'UK', 'MG': 'UK', 'FIAT': 'Italy', 'ALFA ROMEO': 'Italy', 'LANCIA': 'Italy', 'MASERATI': 'Italy', 

In [12]:
# Create new column for country of origin and map it based on manufacturer
df['Country'] = df['Manufacturer'].map(manufacturer_to_country)

In [13]:
# encode categorical column into seperate features
variable = 'Country'
dummies = pd.get_dummies(df[variable]).astype(int)
df = pd.concat([df, dummies], axis=1).drop(columns=variable)

In [14]:
# Finally drop the manufacturer column, model, "other".
# Model column has almost 1600 distinct values and is basically just a name for the car, pretty similar to ID column
# I think there is no way to get any information out of model

df.drop(["Manufacturer", "Model", "Other"], axis=1, inplace=True)

<h2>Category</h2>

Here I would like to join the cabriolet and coupe categories because cabriolet is basically coupe without roof.
Limousine category will be deleted before encoding for the same reason as "other" column previously.

In [15]:
df["Category"].value_counts()

Category
Sedan          8600
Jeep           5378
Hatchback      2799
Minivan         633
Coupe           528
Universal       361
Microbus        299
Goods wagon     229
Pickup           51
Cabriolet        35
Limousine        11
Name: count, dtype: int64

In [16]:
# Replace cabriolet to coupe
df["Category"].replace("Cabriolet", "Coupe", inplace=True)

C:\Users\ksobolev\AppData\Local\Temp\ipykernel_19780\487454773.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Category"].replace("Cabriolet", "Coupe", inplace=True)


In [17]:
# encode categorical column into seperate features
variable = 'Category'
dummies = pd.get_dummies(df[variable]).astype(int)
df = pd.concat([df, dummies], axis=1).drop(columns=variable)

In [18]:
# Drop limousine to decrease multidimensionality
df.drop("Limousine", axis=1, inplace=True)

<h2>Leather interior</h2>
This one is simple, it's yes or no column so just encode it to binary. 

In [19]:
variable = 'Leather interior'

# NOTE: factorize can mix up the order of the values
values1, names1 = pd.factorize(df[variable], sort=False)
df[variable] = values1

<h2>Fuel type</h2>
I think about joining the plug in hybrid and hybrid into one category, they are slightly different but however
Also I think I'll delete row with hydrogen fuel type.

In [20]:
df["Fuel type"].value_counts()

Fuel type
Petrol            9944
Diesel            4001
Hybrid            3539
LPG                885
CNG                469
Plug-in Hybrid      85
Hydrogen             1
Name: count, dtype: int64

In [21]:
# search for the index of the row with fuel type hydrogen
i = df[df["Fuel type"] == "Hydrogen"].index
# Drop the hydrogen
df.drop(i, inplace=True)


In [22]:
# join hybrids
df.replace("Plug-in Hybrid", "Hybrid", inplace=True)

In [23]:
# Encode column and drop CNG to decrease column amount
variable = 'Fuel type'
dummies = pd.get_dummies(df[variable]).astype(int)
df = pd.concat([df, dummies], axis=1).drop(columns=variable)

In [24]:
df.drop("CNG", axis=1, inplace=True)

<h2>Engine volume</h2>

Engine volume consist of float number indicating volume and sometimes also mention if it is turbo or not. 

I can extract volume and seperate turbo feature to different column

In [25]:
# separate number and Turbo
def process_engine(engine_value):
    # Extract the numeric part using regex
    # \d means any number between 0-9
    number = re.search(r'\d+\.\d+|\d+', engine_value)
    # save our extracted float
    number = float(number.group())

    # Check if "Turbo" is in the string
    turbo = 1 if "Turbo" in engine_value else 0

    return number, turbo

# Apply the function to the DataFrame
df[['Engine volume int', 'Turbo']] = df['Engine volume'].apply(lambda x: pd.Series(process_engine(x)))

# Drop the old Engine column
df.drop('Engine volume', axis=1, inplace=True)


<h2>Mileage</h2>
Mileage values are strings and contain "km".

In [26]:
# using regex exctract only numeric values
df['Mileage'].replace(r'[^0-9]+', '', regex=True, inplace=True)
# change data type of column to numeric
df["Mileage"] = pd.to_numeric(df["Mileage"])

C:\Users\ksobolev\AppData\Local\Temp\ipykernel_19780\3909022585.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Mileage'].replace(r'[^0-9]+', '', regex=True, inplace=True)


<h2>Gear box type</h2>
Same as other categorical variables

In [27]:
# Encode column and drop last feature to decrease column amount
variable = 'Gear box type'
dummies = pd.get_dummies(df[variable]).astype(int)
df = pd.concat([df, dummies], axis=1).drop(columns=variable)

df.drop("Tiptronic", axis=1, inplace=True)


<h2>Drive wheels</h2>
Same as previous

In [28]:
# Encode column and drop last feature to decrease column amount
variable = 'Drive wheels'
dummies = pd.get_dummies(df[variable]).astype(int)
df = pd.concat([df, dummies], axis=1).drop(columns=variable)

df.drop("Rear", axis=1, inplace=True)

<h2>Doors</h2>
Column is in wrong format, I assume the real meaning is 4-5 (May is 5th month) for the most part of the cars, 2-3 (March is 3rd) for coupes and >5 for cars with more than 5 doors. 

However, basically amount of doors is same thing as type of car, most of the cars are sedans, universals or hatchabacks and they all have 4 to 5 doors. Cabriolets, coupes, pickups and good wagons have 2 to 3 doors. Buses have more than 5.

Another problem is that this column is very unbalanced, since owerhelming amount of cars have 4-5 doors.


So in this case I assume that is better to drop this column.

In [29]:
df["Doors"].value_counts()

Doors
04-May    18031
02-Mar      768
>5          124
Name: count, dtype: int64

In [30]:
df.drop("Doors", axis=1, inplace=True)

<h2>Wheel</h2>
Defines if the wheel is on right or left side. Binary caregory

In [31]:
variable = 'Wheel'

# NOTE: factorize can mix up the order of the values
values1, names1 = pd.factorize(df[variable], sort=False)
df[variable] = values1

<h2>Color</h2>

Color could be meaningfull for example for luxury cars, may be some kind of rare color combinations and so on. 

Also, when buying a new car in case you want any specific and dealership doesn't have it at the moment you have to wait for it.

But in general I don't think that the color has some significant influence on price, especially on used car market.

In [32]:
df.drop("Color", axis=1, inplace=True)

Next step is to split the dataset for VIF and K best tests

In [33]:
# y - target variable; X - others
X = df.drop("Price", axis=1)
y = df['Price']
# Split 70% for training and 30% for testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
# Split the testing data, half for validation and half for testing
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

<h2>Optimizing dataset</h2>


In [34]:
from ydata_profiling import ProfileReport
import pandas as pd

# Generate the data profiling report, title is just 
report = ProfileReport(df, title='Car price dataset')
report.to_file("my_report.html")

c:\Users\ksobolev\Documents\github\school\Deep-Learning\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Summarize dataset:  89%|████████▉ | 40/45 [00:00<00:00, 51.19it/s, Calculate auto correlation]         c:\Users\ksobolev\Documents\github\school\Deep-Learning\.venv\lib\site-packages\ydata_profiling\model\pandas\discretize_pandas.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  discretized_df.loc[:, column] = self._discretize_column(
c:\Users\ksobolev\Documents\github\school\Deep-Learning\.venv\lib\site-packages\ydata_profiling\model\pandas\discretize_pandas.py:52: FutureWarning: Setting an item of incompatible dtype is depre

For some reason, even though I cleaned the original dataset from duplicates, now there are almost 3000 of duplicates.

It might be that in the original dataset there were cars with for example different colors, but otherwise same.

In [35]:
#
print(f"Amount of deleted duplicate rows: {df.duplicated().sum()}")
df.drop_duplicates(inplace=True)


Amount of deleted duplicate rows: 3236


In [36]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# convert all continuous variables to integer,
# and convert all negative numbers to 0
X_cat = X.astype(int)
X_cat = X_cat.clip(lower=0)

# initialize chi2 and SelectKBest
# Note: chi2 -test is a very common test
# in statistics and quantitative analysis
# basically it studies the data whether variables are related
# or independent of each other
chi_2_features = SelectKBest(chi2, k=len(X_cat.columns))

# fit our data to the SelectKBest
best_features = chi_2_features.fit(X_cat,y.astype(int))

# use decimal format in table print later
pd.options.display.float_format = '{:.2f}'.format

# wrap it up, and show the results
# the higher the score, the more effect that column has on price
df_features = pd.DataFrame(best_features.scores_)
df_columns = pd.DataFrame(X_cat.columns)
f_scores = pd.concat([df_columns,df_features],axis=1)
f_scores.columns = ['Features','Score']
f_scores.sort_values(by='Score',ascending=False)

,Features,Score
3,Mileage,1743236078567.59
0,Levy,2454171.21
6,Airbags,13906.85
13,South Korea,6830.25
22,Minivan,6277.54
25,Universal,6150.55
26,Diesel,5706.60
2,Leather interior,4555.59
5,Wheel,4226.19
28,LPG,4034.58


In [37]:
# pip install statsmodels
from statsmodels.stats.outliers_influence import variance_inflation_factor 

# VIF dataframe 
# VIF = Variance Inflation Factor
vif_data = pd.DataFrame() 
vif_data["feature"] = X.columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(X.values, i) 
                          for i in range(len(X.columns))] 
  

# variables with high VIF-value 
# can mean multlicollinearity (variables providing same linear
# relationships in the data, potentially confusing the ML algorithm
# this might be good info when deciding if some variable needs to be removed
print(vif_data)

              feature     VIF
0                Levy    3.28
1          Prod. year 2710.50
2    Leather interior    2.30
3             Mileage    1.01
4           Cylinders   45.86
5               Wheel    1.53
6             Airbags    4.39
7               China    1.08
8              France    3.58
9             Germany  183.66
10              Italy    4.72
11              Japan  297.70
12             Russia    4.19
13        South Korea  199.60
14             Sweden    1.88
15                 UK    7.09
16                USA  104.88
17              Coupe   53.43
18        Goods wagon   22.55
19          Hatchback  263.78
20               Jeep  504.57
21           Microbus   29.11
22            Minivan   60.28
23             Pickup    5.79
24              Sedan  805.01
25          Universal   34.84
26             Diesel   11.09
27             Hybrid   10.46
28                LPG    3.23
29             Petrol   24.65
30  Engine volume int   27.51
31              Turbo    1.54
32        

Another thing that I've noticed is that Production Year doesn't correlate with price, has very low K-best score and high VIF score (which might mean that production year and mileage are depandant variables)

In [38]:
df.drop("Prod. year", axis=1, inplace=True)

In [39]:
# process the whole dataset 
# and compares variables to each other instead
# of focusing on outliers on a single variable
from sklearn.ensemble import IsolationForest
 
iso = IsolationForest(contamination=0.05) 

# fit isolation forest
y_pred = iso.fit_predict(df)

# filter outliers only
outliers = df[y_pred != 1]

In [40]:
outliers

,Price,Levy,Leather interior,Mileage,Cylinders,Wheel,Airbags,China,France,Germany,...,Hybrid,LPG,Petrol,Engine volume int,Turbo,Automatic,Manual,Variator,4x4,Front
11,8781,0,1,0,8.00,0,0,0,0,0,...,0,0,0,4.00,0.00,0,1,0,0,0
23,7840,0,1,230000,4.00,0,0,0,0,0,...,0,0,0,2.00,1.00,0,1,0,0,1
30,15681,1288,1,180000,6.00,0,4,0,0,1,...,0,0,0,2.00,1.00,0,1,0,0,0
34,24462,0,0,250000,6.00,0,10,0,0,0,...,0,0,0,3.00,1.00,0,0,0,1,0
49,17249,0,1,147000,4.00,0,8,0,0,0,...,1,0,0,2.30,0.00,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19161,64290,0,0,97000,6.00,0,12,0,0,0,...,1,0,0,3.50,0.00,0,0,1,1,0
19167,18817,1995,1,2147483647,4.00,0,2,0,0,0,...,0,0,0,2.40,1.00,0,1,0,0,1
19194,500,0,1,760000,4.00,0,2,0,0,0,...,0,0,0,2.50,0.00,0,1,0,0,0
19205,7500,0,1,18500,4.00,1,2,0,0,0,...,0,0,1,2.00,0.00,0,0,1,1,0


In [41]:
df.drop(outliers.index, inplace=True)

And split again

In [42]:
# y - target variable; X - others
X = df.drop("Price", axis=1)
y = df['Price']
# Split 70% for training and 30% for testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
# Split the testing data, half for validation and half for testing
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

<h2>Building model</h2>

<h2>Training</h2>

In [43]:
# pip install keras-tuner
import keras_tuner

def build_model(hp):
    # iniatlize sequential test neural network
    model = keras.Sequential()

    # first layer, batch normalization + input shape, same as in typical neural network
    #model.add(layers.BatchNormalization(input_shape=(len(X.columns),)),)
    model.add(
        layers.Dense(input_shape=(len(X.columns),),
            # Tune number of units separately.
            units=hp.Int(f"imput", min_value=32, max_value=128, step=4),
            activation=hp.Choice("activation", ["relu"]),
        )
    )
    # add the first actual layer including the regularizer
    model.add(
        layers.Dense(
            # Tune number of units separately.
            units=hp.Int(f"units_0", min_value=32, max_value=86, step=4),
            activation=hp.Choice("activation", ["relu"]),
            kernel_regularizer=keras.regularizers.l1(l1=0.1)
        )
    )
        
    # automate a dropout layer
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.1))

    # try additional layers, 1 or 2 extra layers
    for i in range(hp.Int("num_layers", 1, 2)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i + 1}", min_value=4, max_value=64, step=4),
                activation=hp.Choice("activation", ["relu"]),
            )
        )

    # output layer, only one node since this is regression
    model.add(layers.Dense(1))

    # automate learning rate tests
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")

    # compile the test neural network
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mse"
    )

    return model


# build the model + use RandomSearch to actually search the best options for our neural network
build_model(keras_tuner.HyperParameters())

# use val_loss as the objective, because regression tasks do not have accuracy
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=3,
    overwrite=True,
    directory="optimizations",
    project_name="regression1test",
)

# start searching
#tuner.search(X_train, y_train, epochs=250, validation_data=(X_val, y_val))

ModuleNotFoundError: No module named 'keras_tuner'

In [36]:
#tuner.results_summary()

Results summary
Results in optimizations\regression1test
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 02 summary
Hyperparameters:
imput: 108
activation: relu
units_0: 84
dropout: False
num_layers: 2
units_1: 20
lr: 0.0014601330934395126
units_2: 56
Score: 243576968533.33334

Trial 04 summary
Hyperparameters:
imput: 36
activation: relu
units_0: 60
dropout: False
num_layers: 1
units_1: 44
lr: 0.002828685755041001
units_2: 52
Score: 243584548864.0

Trial 01 summary
Hyperparameters:
imput: 84
activation: relu
units_0: 84
dropout: False
num_layers: 1
units_1: 16
lr: 0.0004591527984583266
units_2: 4
Score: 243624340138.66666

Trial 09 summary
Hyperparameters:
imput: 72
activation: relu
units_0: 56
dropout: False
num_layers: 2
units_1: 44
lr: 0.0031956353244411316
units_2: 56
Score: 243634623829.33334

Trial 06 summary
Hyperparameters:
imput: 64
activation: relu
units_0: 76
dropout: True
num_layers: 1
units_1: 48
lr: 0.0003072922963104916
units_2: 16
Score: 243676

In [ ]:
#models = tuner.get_best_models(num_models=2)
#best_model = models[0]
#best_model.summary()

In [44]:
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu", input_shape=(len(X.columns),)),
        layers.Dense(32, activation="relu", kernel_regularizer=keras.regularizers.l2(l2=0.1)),
        layers.Dense(16, activation="relu", kernel_regularizer=keras.regularizers.l2(l2=0.1)),
        layers.Dense(8, activation="relu"),
        layers.Dense(1)
    ]
)

# optimal learning rate from keras tuner
optimal_lr = 0.0014601330934395126

# select the optimizer and loss function
# you can try rmsprop also as optimizer, or stochastic gradient descent
model.compile(optimizer=keras.optimizers.Adam(), loss='mse')
# print out the summary of your model
model.summary()

c:\Users\ksobolev\Documents\github\school\Deep-Learning\.venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         2,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,121 (20.00 KB)

 Trainable params: 5,121 (20.00 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
model.fit(x=X_train, y=y_train, epochs=800, validation_data=(X_val, y_val))

Epoch 1/800
326/326 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 4787960545280.0000 - val_loss: 693901504.0000
Epoch 2/800
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13753870450688.0000 - val_loss: 521558592.0000
Epoch 3/800
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 508360818688.0000 - val_loss: 513764576.0000
Epoch 4/800
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4745439739904.0000 - val_loss: 528119456.0000
Epoch 5/800
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 769119872.0000 - val_loss: 484988288.0000
Epoch 6/800
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 72638431232.0000 - val_loss: 474999136.0000
Epoch 7/800
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 455555072.0000 - val_loss: 457142432.0000
Epoch 8/800
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 436737056.0000 - val_loss: 444251040.0000
Epoch 9/800
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 686581440.0000 - val_loss: 435208288.0000
Epoch 10/800
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step -

<h2>Metrics</h2>

In [46]:
# try to look if the model is actually training 
# => the error is going downwards
# if using validation data, you get two lines
# in this case, see if the lines follow a similar trend 
# (they don't always overlap with complex data, the trend is more important)
loss_df = pd.DataFrame(model.history.history)
loss_df.plot()

# other notes:
# if your validation loss is fluctuating a lot, the test data set
# might not be a good sample / representation of the whole dataset
# try to get more data or try shuffling the dataset for a better sample
# if your validation loss FLUCTUATES EXTREMELY: remember to create 
# the neural network again completely (Sequential etc.), otherwise you might fit your 
# model to your previous model version weights

# we'll also study later methods on how to select the best epoch from 
# the training history

# a common reason why a numeric neural network for regression might overfit
# is that you might have too many variables but too little data

<Axes: >

In [47]:
# compare test error values to training error values
# the model is often good when these error values are similar
# even if you training metrics above didn't overlap
# you might still get very close values in evaluation => more important

# compare the final model loss/evaluation values
print("Test data evaluation:")
print(model.evaluate(X_test, y_test, verbose=0))
print("\nTrain data evaluation:")
print(model.evaluate(X_train, y_train, verbose=0))

Test data evaluation:
598171328.0

Train data evaluation:
575705920.0


In [48]:
test_predictions = model.predict(X_test)

# reshape the data for easier comparison table
test_predictions = pd.Series(test_predictions.reshape(len(y_test),))
pred_df = pd.DataFrame(np.asarray(y_test), columns=['Test True Y'])
pred_df = pd.concat([pred_df, test_predictions], axis=1)
pred_df.columns = ['Test True Y', 'Model Predictions']

# print the comparison table - true values vs. model predicted values
# we can nicely see here how far off our model is in some cases
pred_df

70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


,Test True Y,Model Predictions
0,14269,155.09
1,31047,155.09
2,5645,155.09
3,4704,155.09
4,26971,155.09
...,...,...
2231,99440,155.09
2232,10036,155.09
2233,22843,155.09
2234,2509,155.09


In [46]:
# these values follow a linear line = good predictions
# we basically compare the predicted values 
# to true test values and see the differences
sns.scatterplot(x='Test True Y', y='Model Predictions', data=pred_df)

<Axes: xlabel='Test True Y', ylabel='Model Predictions'>

In [49]:
# MAE - Mean average error
print("MAE")
print(round(metrics.mean_absolute_error(y_test, test_predictions), 2), "$")

# MSE - Mean square error
print("\nMSE")
print(round(metrics.mean_squared_error(y_test, test_predictions), 2), "$^2")

# RMSE - Root mean square error
print('\nRMSE:')
print(round(np.sqrt(metrics.mean_squared_error(y_test, test_predictions)), 2), "$")

# R-squared. 0 = the model descibes the dataset poorly
# 1 = model describes the dataset perfectly
print('\nR-squared:')
print(round(metrics.r2_score(y_test, test_predictions), 2))

# Explained Variance Score => 0 = the model descibes the dataset poorly
# 1 = model describes the dataset perfectly
# high variance score = model is a good fit for the data 
# low variance score = model is not a good fit for the data
# the higher the score, the model is more able to explain the variation in the data
# if score is low, we might need more and better data
print("\nExplained variance score:")
print(round(metrics.explained_variance_score(y_test, test_predictions), 2))

MAE
17885.24 $

MSE
598168820.3 $^2

RMSE:
24457.49 $

R-squared:
-1.15

Explained variance score:
-0.03


In [48]:
# if the prediction distribution are far from normal distribution
# then the model is not probably good enough
# distplot is deprecating in future pandas-version
# unfortunately, there's no exact alternative to do this plot at the moment
sns.distplot((y_test - test_predictions))
plt.show()
plt.close()

C:\Users\ksobolev\AppData\Local\Temp\ipykernel_9980\3124900743.py:5: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot((y_test - test_predictions))
C:\Users\ksobolev\AppData\Local\Temp\ipykernel_9980\3124900743.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
